In [1]:
# Import necessary libraries
import sqlite3
import pandas as pd
from rdflib import Graph, Namespace
from open_fdd.air_handling_unit.faults import FaultConditionOne

In [2]:
conn = sqlite3.connect("brick_timeseries.db")

In [3]:
# Configuration dictionary template
PERCENTAGE_COLS_TO_CONVERT = [
    "Supply_Fan_VFD_Speed_Sensor",  # BRICK formatted column name
]

In [4]:
config_dict_template = {
    "INDEX_COL_NAME": "timestamp",
    "DUCT_STATIC_COL": "Supply_Air_Static_Pressure_Sensor",
    "DUCT_STATIC_SETPOINT_COL": "Supply_Air_Static_Pressure_Setpoint",
    "SUPPLY_VFD_SPEED_COL": "Supply_Fan_VFD_Speed_Sensor",
    "VFD_SPEED_PERCENT_ERR_THRES": 0.05,
    "VFD_SPEED_PERCENT_MAX": 0.99,
    "DUCT_STATIC_INCHES_ERR_THRES": 0.1,
    "TROUBLESHOOT_MODE": False,
    "ROLLING_WINDOW_SIZE": 10,
}

In [5]:
def run_sparql_query(graph):
    print("Running SPARQL query...")
    query = """
    PREFIX brick: <https://brickschema.org/schema/Brick#>
    PREFIX ref: <https://brickschema.org/schema/Reference#>

    SELECT ?ahu ?sensorType ?sensor WHERE {
        ?ahu brick:hasPoint ?sensor .
        ?sensor a ?sensorType .
        FILTER (?sensorType IN (brick:Supply_Air_Static_Pressure_Sensor, brick:Supply_Air_Static_Pressure_Setpoint, brick:Supply_Fan_VFD_Speed_Sensor))
    }
    """
    return graph.query(query)


# Function to load RDF graph
def load_rdf_graph(file_path):
    print("Loading RDF graph...")
    g = Graph()
    g.parse(file_path, format="turtle")
    return g


# Function to convert analog output columns to floats
def convert_floats(df, columns):
    for column in columns:
        df[column] = df[column] / 100.0
    return df


# Function to run fault condition one
def run_fault_one(config_dict, df):
    fc1 = FaultConditionOne(config_dict)
    df = fc1.apply(df)
    return df


# Function to retrieve time series data
def retrieve_timeseries_data(sensor_data, conn):
    dfs = []
    for ahu, sensors in sensor_data.items():
        print(f"Querying SQLite for AHU: {ahu}")
        df_ahu = None
        for sensor_type, sensor_uri in sensors.items():
            sensor_id = sensor_uri.split("/")[-1]
            print(f"Querying SQLite for sensor: {sensor_id} of type: {sensor_type}")
            sql_query = """
            SELECT timestamp, value
            FROM TimeseriesData
            WHERE sensor_name = ?
            """
            df_sensor = pd.read_sql_query(sql_query, conn, params=(sensor_id,))
            if df_sensor.empty:
                print(
                    f"No data found for sensor: {sensor_type} with sensor_id: {sensor_id}"
                )
            else:
                print(
                    f"Data found for sensor: {sensor_type}, number of records: {len(df_sensor)}"
                )
                df_sensor = df_sensor.rename(columns={"value": sensor_type})
                if df_ahu is None:
                    df_ahu = df_sensor.set_index("timestamp")
                else:
                    df_ahu = pd.merge(
                        df_ahu,
                        df_sensor.set_index("timestamp"),
                        left_index=True,
                        right_index=True,
                    )
        if df_ahu is not None:
            dfs.append((ahu, df_ahu))
    return dfs

In [6]:
# Load the RDF graph
rdf_graph = load_rdf_graph("brick_model_with_timeseries.ttl")

rdf_graph

Loading RDF graph...


<Graph identifier=Ne07647bc95744620ba9d868622412882 (<class 'rdflib.graph.Graph'>)>

In [7]:
# Run SPARQL query to get sensor data for each AHU
sparql_results = run_sparql_query(rdf_graph)

sparql_results

Running SPARQL query...


In [8]:
# Extract sensor data from the SPARQL query result
sensor_data = {}
for row in sparql_results:
    ahu = str(row.ahu).split("/")[-1]
    sensor_type = str(row.sensorType).split("#")[-1]
    sensor_data.setdefault(ahu, {})[sensor_type] = row.sensor
    print(f"Found sensor for {ahu}: {sensor_type} -> {row.sensor}")

# Retrieve time series data for each AHU
ahu_dataframes = retrieve_timeseries_data(sensor_data, conn)

Found sensor for AHU1: Supply_Air_Static_Pressure_Setpoint -> http://example.org/AHU1_Eff_StaticSPt
Found sensor for AHU1: Supply_Fan_VFD_Speed_Sensor -> http://example.org/AHU1_SaFanSpeedAO_value
Found sensor for AHU1: Supply_Air_Static_Pressure_Sensor -> http://example.org/AHU1_SaStatic_value
Found sensor for AHU2: Supply_Fan_VFD_Speed_Sensor -> http://example.org/AHU2_SaFanSpeedAO_value
Found sensor for AHU2: Supply_Air_Static_Pressure_Sensor -> http://example.org/AHU2_SaStatic_value
Found sensor for AHU2: Supply_Air_Static_Pressure_Setpoint -> http://example.org/AHU2_StaticSPt
Found sensor for AHU3: Supply_Fan_VFD_Speed_Sensor -> http://example.org/AHU3_SaFanSpeedAO_value
Found sensor for AHU3: Supply_Air_Static_Pressure_Sensor -> http://example.org/AHU3_SaStatic_value
Found sensor for AHU3: Supply_Air_Static_Pressure_Setpoint -> http://example.org/AHU3_StaticSPt
Found sensor for AHU4: Supply_Fan_VFD_Speed_Sensor -> http://example.org/AHU4_SaFanSpeedAO_value
Found sensor for AHU4: 

In [9]:
ahu_dataframes_dict = {}

# Loop through each AHU's DataFrame for processing
for ahu, df_combined in ahu_dataframes:
    print(f"Processing data for AHU: {ahu}")

    # Convert analog outputs to floats
    df_combined = convert_floats(df_combined, PERCENTAGE_COLS_TO_CONVERT)

    # Customize the config_dict for each AHU if necessary
    config_dict = config_dict_template.copy()

    # Run the fault detection function
    try:
        df_combined = run_fault_one(config_dict, df_combined)
        print(f"Total faults detected for {ahu}: {df_combined['fc1_flag'].sum()}")

        # Store the DataFrame in a dictionary
        ahu_dataframes_dict[ahu] = df_combined
    except TypeError as e:
        print(f"Error processing AHU {ahu}: {e}")

Processing data for AHU: AHU1
Total faults detected for AHU1: 0
Processing data for AHU: AHU2
Total faults detected for AHU2: 0
Processing data for AHU: AHU3
Total faults detected for AHU3: 1792
Processing data for AHU: AHU4
Total faults detected for AHU4: 7828


In [10]:
# Loop through the dictionary to access each AHU's DataFrame
for ahu_name, df in ahu_dataframes_dict.items():
    # Print the AHU name
    print(f"AHU: {ahu_name}")

    # Print the total number of flags detected
    total_flags = df["fc1_flag"].sum()
    print(f"Total fc1_flags detected: {total_flags}")

    # Print a small sample of the DataFrame (e.g., first 5 rows)
    # print("Data sample:")
    # print(df.head())  # You can adjust the number of rows with df.head(n)

    # Optionally, print a line separator for readability
    print("-" * 40)

AHU: AHU1
Total fc1_flags detected: 0
----------------------------------------
AHU: AHU2
Total fc1_flags detected: 0
----------------------------------------
AHU: AHU3
Total fc1_flags detected: 1792
----------------------------------------
AHU: AHU4
Total fc1_flags detected: 7828
----------------------------------------


In [11]:
conn.close()